In [1]:
import pandas as pd

from data import loader
from utils import supply_chain, vrp, warehouse

In [2]:
demand_df, prices_df, polygon_df = loader.create_datasets()

In [3]:
demand_df

,specie,polygon,demand
0,Agave lechuguilla,2,248
1,Agave salmiana,2,1179
2,Agave scabra,2,248
3,Agave striata,2,248
4,Opuntia cantabrigiensis,2,293
5,Opuntia engelmani,2,225
6,Opuntia robusta,2,436
7,Opuntia streptacanta,2,383
8,Prosopis laevigata,2,518
9,Yucca filifera,2,158


In [4]:
warehouse = warehouse.Warehouse(
    # max_capacity = 4000,
    max_capacity=4_000,
    min_storage_days = 3
)

vrp_optimizer = vrp.VRP(
    polygon_df = polygon_df,
    hq_polygon = 18,
    # truck_capacity = 542,
    truck_capacity = 524,
    loading_time = 30,
    unloading_time = 30,
    max_time = 360,
    speed_kmh = 10.0
)

sc_optimizer = supply_chain.SASCOpt(
    demand_df = demand_df,
    prices_df = prices_df,
    polygon_df = polygon_df,
    max_load = 8_000,
    transport_cost = 4_500,
    iterations = 10,
    initial_temp = 1_000,
    cooling_rate = 0.995,
    log_file = './output/SA_Supply_Chain.log',
    warehouse = warehouse,
    vrp_optimizer = vrp_optimizer,
    utilization_threshold = 0.75,
    utilization_penalty_scale = 1_000_000
)

In [5]:
best_solution, best_cost, supply_chain_json, vrp_actions_json = sc_optimizer.run()

with open("./scheduling/supply_chain_schedule.json", "w") as f:
    f.write(supply_chain_json)

with open("./scheduling/vrp_actions.json", "w") as f:
    f.write(vrp_actions_json)

100%|██████████| 10/10 [00:03<00:00,  2.68it/s]


In [6]:
sc_optimizer.scheduling_cost(best_solution)

459601.0

In [7]:
best_cost

9455601.0

In [8]:
best_solution

[(1, 18, 'Agave salmiana', 'Laguna seca', 1115),
 (1, 3, 'Agave scabra', 'Laguna seca', 264),
 (1, 4, 'Agave scabra', 'Moctezuma', 264),
 (1, 3, 'Agave salmiana', 'Laguna seca', 1254),
 (1, 3, 'Opuntia robusta', 'Venado', 463),
 (1, 3, 'Agave striata', 'Moctezuma', 264),
 (1, 4, 'Agave salmiana', 'Laguna seca', 376),
 (5, 18, 'Opuntia robusta', 'Laguna seca', 412),
 (5, 2, 'Opuntia streptacanta', 'Venado', 383),
 (5, 2, 'Opuntia robusta', 'Laguna seca', 436),
 (5, 18, 'Agave striata', 'Moctezuma', 234),
 (5, 3, 'Agave lechuguilla', 'Laguna seca', 53),
 (6, 3, 'Opuntia streptacanta', 'Venado', 408),
 (6, 18, 'Opuntia streptacanta', 'Venado', 258),
 (7, 2, 'Agave salmiana', 'Laguna seca', 502),
 (8, 3, 'Prosopis laevigata', 'Vivero', 551),
 (8, 4, 'Opuntia robusta', 'Laguna seca', 463),
 (8, 18, 'Opuntia engelmani', 'Venado', 213),
 (8, 3, 'Yucca filifera', 'Vivero', 87),
 (9, 2, 'Prosopis laevigata', 'Vivero', 518),
 (9, 4, 'Opuntia streptacanta', 'Venado', 408),
 (9, 4, 'Prosopis laevi